<a href="https://colab.research.google.com/github/nbadiuc/Automatic-Text-Simplification-Based-On-NER/blob/main/Thesis_Automatic_Text_Simplification_Based_On_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installations and imports

In [ ]:
!pip install torch
!pip install transformers
!pip install xformers
!pip install sentencepiece
!pip install sacremoses

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
from transformers import pipeline

import torch
from sklearn.metrics.pairwise import cosine_distances
import pandas as pd
import zipfile
import os
import string
import requests
import json
import random
import re

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
!pip install pymorphy2
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# The baseline model

In [ ]:
baseline_tokenizer = AutoTokenizer.from_pretrained("orzhan/rugpt3-simplify-large", padding_side='left')
baseline_model = AutoModelForCausalLM.from_pretrained("orzhan/rugpt3-simplify-large")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
baseline_simplifier = pipeline("text-generation", model=baseline_model, tokenizer=baseline_tokenizer, max_length=150)

In [ ]:
def base_model(sentence):
    sentence = "<s>" + sentence + "<Simplify:>"
    while sentence.find("EndText") == -1:
        sentence = baseline_simplifier(sentence)[0]['generated_text']
    return sentence[sentence.find("<Simplify:>")+12:sentence.find("EndText")-1]

In [ ]:
print(base_model("Финансовым директором социальной сети Facebook назначен 39-летний Дэвид Эберсман (David Ebersman), сообщает The Wall Street Journal."))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Дэвид Эберсман (David Ebersman) - финансовый директор социальной сети Facebook.


#BERT and synonym dictionary models

In [ ]:
# download the dataset into the /tmp folder
!wget "https://github.com/nerel-ds/NEREL/archive/refs/heads/master.zip" -O "/tmp/nerel.zip"

zip_ref = zipfile.ZipFile('/tmp/nerel.zip', 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

--2023-06-04 08:42:51--  https://github.com/nerel-ds/NEREL/archive/refs/heads/master.zip
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/nerel-ds/NEREL/zip/refs/heads/master [following]
--2023-06-04 08:42:51--  https://codeload.github.com/nerel-ds/NEREL/zip/refs/heads/master
Resolving codeload.github.com (codeload.github.com)... 192.30.255.120
Connecting to codeload.github.com (codeload.github.com)|192.30.255.120|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘/tmp/nerel.zip’

/tmp/nerel.zip          [    <=>             ]   6.14M  8.14MB/s    in 0.8s    

2023-06-04 08:42:52 (8.14 MB/s) - ‘/tmp/nerel.zip’ saved [6443546]



In [ ]:
# reading a text and its annotation
def read_text(text_location="/tmp/NEREL-master/NEREL-v1.1/train/023.txt"):

    with open(text_location) as f:
        lines = f.readlines()
    
    # getting rid of the titles of the articles - they are not full sentences
    offset = len(lines[0]) # title length

    text = ''
    for i in range(1, len(lines)):
        text += lines[i]

    ann_location = text_location[:-3] + "ann"
    with open(ann_location) as f:
        ann = f.readlines()

    return text, ann, offset

In [79]:
text, ann, offset = read_text(text_location="/tmp/NEREL-master/NEREL-v1.1/train/023.txt")
print("Text:")
print(text)
print("Annotation:")
print(ann)
print("Offset:")
print(offset)

Text:
Финансовым директором социальной сети Facebook назначен 39-летний Дэвид Эберсман (David Ebersman), сообщает The Wall Street Journal. На работу в Facebook он выйдет в сентябре. Ранее Эберсман был финансовым директором биотехнологической компании Genentech.

Эберсман подчеркнул, что видит много общего между Facebook и Genentech. В частности, это две быстрорастущие компании с сильной корпоративной культурой. Также он заявил, что Facebook ожидает 70-процентное увеличение выручки в 2009 году.

В компании Genentech Дэвид Эберсман проработал 15 лет. Ее финансовым директором он стал в 2006 году. На этой должности Эберсман проработал до апреля 2009 года, когда Roche Holding купил Genentech.

По данным The Wall Street Journal, на должность финансового директора Facebook претендовали 11 кандидатов. Каждый из них прошел собеседование с основателем Facebook Марком Цукербергом (Mark Zuckerberg) и другими руководителями компании.

Ранее финансовым директором Facebook был Гидеон Ю (Gideon Yu). О

In [80]:
# using annotation, generates a mask that contains 1s in the places corresponding to named entities and 0s otherwise
def generate_NE_mask(text, ann, offset):
    mask = [0] * len(text)
    for line in ann:
        if line[0] == 'T': # "T" - terms; in annotation, they correspond to named entites
            if ';' in line:
                line = line.split(';')[0]
            start_ind, end_ind = line.split('\t')[1].split(' ')[1:3]
            #named_entity = line.split('\t')[2].strip()
            start_ind = int(start_ind) - offset
            end_ind = int(end_ind) - offset
            if start_ind >= 0:
                for i in range(start_ind, end_ind):
                    mask[i] = 1
    return mask

In [81]:
mask = generate_NE_mask(text, ann, offset)
for i in range(min(len(mask), 250)):
    print(mask[i], end='')
print()
print(text[:min(len(mask), 250)])

1111111111111111111110000000000000000011111111011111111011111111101111111111111100111111111111110000000000001111111111111111111111100000000000000111111110000000000011111111110000000011111111000001111111111111111111110000000000000000000000000000011111
Финансовым директором социальной сети Facebook назначен 39-летний Дэвид Эберсман (David Ebersman), сообщает The Wall Street Journal. На работу в Facebook он выйдет в сентябре. Ранее Эберсман был финансовым директором биотехнологической компании Genen


In [ ]:
# splits text into sentences remembering start positions of each one
# (relative to text that does not contain name of the article, so, to match the positions in annotation, you need to add offset)
def split_text_into_sentences(text):
    sentences = sent_tokenize(text, language="russian")
    sentence_start_positions = []
    for i in range(len(sentences)):
        sentence_start_positions.append(text.find(sentences[i]))
    return sentences, sentence_start_positions

In [82]:
sentences, sentence_start_positions = split_text_into_sentences(text)
for i in range(len(sentences)):
    print(sentence_start_positions[i], sentences[i])

0 Финансовым директором социальной сети Facebook назначен 39-летний Дэвид Эберсман (David Ebersman), сообщает The Wall Street Journal.
133 На работу в Facebook он выйдет в сентябре.
176 Ранее Эберсман был финансовым директором биотехнологической компании Genentech.
257 Эберсман подчеркнул, что видит много общего между Facebook и Genentech.
329 В частности, это две быстрорастущие компании с сильной корпоративной культурой.
409 Также он заявил, что Facebook ожидает 70-процентное увеличение выручки в 2009 году.
494 В компании Genentech Дэвид Эберсман проработал 15 лет.
549 Ее финансовым директором он стал в 2006 году.
595 На этой должности Эберсман проработал до апреля 2009 года, когда Roche Holding купил Genentech.
692 По данным The Wall Street Journal, на должность финансового директора Facebook претендовали 11 кандидатов.
799 Каждый из них прошел собеседование с основателем Facebook Марком Цукербергом (Mark Zuckerberg) и другими руководителями компании.
930 Ранее финансовым директором 

In [ ]:
# splits sentences into words remembering start position of each one (relative to text without the name of article)
# gets rid of punctuation and stop words while doing so
def split_sentences_into_words(text, sentences, sentence_start_positions):

    words = []
    words_start_positions = []

    for i in range(len(sentences)):
        current_words = word_tokenize(sentences[i], language="russian")
        stop = set(stopwords.words('russian') + list(string.punctuation))
        current_words = [word for word in current_words if word not in stop]

        current_piece_of_text = text[sentence_start_positions[i]:]
        current_words_start_positions = []
        for word in current_words:
            current_words_start_positions.append(current_piece_of_text.find(word) + sentence_start_positions[i])
        words.append(current_words)
        words_start_positions.append(current_words_start_positions)

    return words, words_start_positions

In [83]:
words, words_start_positions = split_sentences_into_words(text, sentences, sentence_start_positions)
print(words)
print(words_start_positions)

[['Финансовым', 'директором', 'социальной', 'сети', 'Facebook', 'назначен', '39-летний', 'Дэвид', 'Эберсман', 'David', 'Ebersman', 'сообщает', 'The', 'Wall', 'Street', 'Journal'], ['На', 'работу', 'Facebook', 'выйдет', 'сентябре'], ['Ранее', 'Эберсман', 'финансовым', 'директором', 'биотехнологической', 'компании', 'Genentech'], ['Эберсман', 'подчеркнул', 'видит', 'общего', 'Facebook', 'Genentech'], ['В', 'частности', 'это', 'две', 'быстрорастущие', 'компании', 'сильной', 'корпоративной', 'культурой'], ['Также', 'заявил', 'Facebook', 'ожидает', '70-процентное', 'увеличение', 'выручки', '2009', 'году'], ['В', 'компании', 'Genentech', 'Дэвид', 'Эберсман', 'проработал', '15', 'лет'], ['Ее', 'финансовым', 'директором', 'стал', '2006', 'году'], ['На', 'должности', 'Эберсман', 'проработал', 'апреля', '2009', 'года', 'Roche', 'Holding', 'купил', 'Genentech'], ['По', 'данным', 'The', 'Wall', 'Street', 'Journal', 'должность', 'финансового', 'директора', 'Facebook', 'претендовали', '11', 'кандида

In [ ]:
word_frequencies = pd.read_csv('/content/drive/My Drive/freqrnc2011.csv', sep='\t', index_col='Lemma')

len_threshold = 7
frequency_threshold = 1/21

def word_complexity(word, method):
    if method == "len":
        return len(word)
    if method == "frequency":
        if word in word_frequencies.index:
            word_frequency = word_frequencies.loc[word]['Freq(ipm)']
            if isinstance(word_frequency, pd.core.series.Series):
                word_frequency = max(word_frequency)
            return 1/word_frequency
        else:
            return 1/20
#            return 15/word_frequency + len(word)/14
#        else:
#            return len(word)/7

In [ ]:
def get_normal_form(word):
    return morph.parse(word)[0].normal_form

In [ ]:
bert_simplifier = pipeline("fill-mask", model="bert-base-multilingual-cased")

def bert_model(initial_sentence, masked_word, word_start_position_in_initial_sentence, complexity_method):#, word_start_position_in_simplified_sentence, simplified_sentence):
    masked_sentence = initial_sentence[:word_start_position_in_initial_sentence] + "[MASK]" + initial_sentence[word_start_position_in_initial_sentence + len(masked_word):]
    results = bert_simplifier(masked_sentence)
    for result in results:
        suggested_word = result['token_str']
        if word_complexity(get_normal_form(suggested_word), complexity_method) < word_complexity(masked_word, complexity_method):
            return suggested_word
    return masked_word 

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def get_synonyms(word):
    url = 'https://dictionary.yandex.net/api/v1/dicservice.json/lookup'
    params = {
        'key': 'dict.1.1.20230528T133010Z.158a1e5336183ca0.4e03e68250b1dfaa01b77acbf3439b0e45d1a614',
        'lang': 'ru-ru',
        'text': word
    }
    response = requests.get(url, params=params)
    data = json.loads(response.text)
    try:
        synonyms_list = []
        for i in data['def'][0]['tr']:
            for j in i['syn']:
                synonyms_list.append(j['text'])
            synonyms_list.append(i['text'])
        return synonyms_list
    except:
        return []

In [ ]:
def get_declension(word):
    p = morph.parse(word)[0]
    tags = set()
    for tag in {p.tag.case, p.tag.gender, p.tag.number, p.tag.tense, p.tag.person, p.tag.aspect, p.tag.involvement, p.tag.mood, p.tag.voice}:
        if tag != None:
            tags.add(tag)
    return tags

In [ ]:
get_declension("красноречивый")

{'masc', 'nomn', 'sing'}

In [ ]:
def decline(word, tags):
    p = morph.parse(word)[0]
    for tag in tags:
        if p.inflect({tag}) != None:
            p = p.inflect({tag})
    return p.word

In [ ]:
def synonym_model(word, complexity_method):
    declension = get_declension(word)
    normalized_word = get_normal_form(word)
    synonyms = get_synonyms(normalized_word)
    #print(word)
    #print(synonyms)
    result = word
    for synonym in synonyms:
        if word_complexity(synonym, complexity_method) < word_complexity(result, complexity_method):
            result = synonym
    result = decline(result, declension)
    if word[0].isupper():
        result = result.capitalize()
    return result

In [ ]:
synonym_model("апробации", "frequency")

'испытания'

In [ ]:
simple_dir = '/content/drive/My Drive/simplifications'
if not os.path.exists(os.path.join(simple_dir, 'original')):
    os.makedirs(os.path.join(simple_dir, 'original'))

def write_list_to_file(list, path): # path where the file is located
    with open(path, 'w') as f:
        for item in list:
            f.write("%s\n" % item)

def read_list_from_file(path):
    with open(path) as f:
        sentences = f.readlines()
    return sentences

In [72]:
def simplify(text_location, model, complexity_method):

    text, ann, offset = read_text(text_location)
    mask = generate_NE_mask(text, ann, offset)
    sentences, sentence_start_positions = split_text_into_sentences(text)
    words, words_start_positions = split_sentences_into_words(text, sentences, sentence_start_positions)

    file_name = text_location.split('/')[-1]
    if not os.path.exists(os.path.join(simple_dir, 'original', file_name)):
        write_list_to_file(sentences, os.path.join(simple_dir, 'original', file_name))
    simplified_sentences = []
    
    for sentence_ind in range(len(sentences)):
        if model == "baseline":
            simplified_sentence = base_model(sentences[sentence_ind])
        else:
            simplified_sentence = sentences[sentence_ind]
            words_start_positions_in_simplified_sentence = words_start_positions[sentence_ind][:]
            for word_ind in range(len(words[sentence_ind])):
                words_start_positions_in_simplified_sentence[word_ind] -= sentence_start_positions[sentence_ind]

            for word_ind in range(len(words[sentence_ind])):
                current_word = words[sentence_ind][word_ind]
                if mask[words_start_positions[sentence_ind][word_ind]] == 0: # if the current word is not part of a named entity
                    normalized_current_word = get_normal_form(current_word)
                    if complexity_method == "len":
                        threshold = len_threshold
                    else:
                        threshold = frequency_threshold
                    if word_complexity(normalized_current_word, complexity_method) > threshold:
                        current_word_start_position_in_initial_sentence = words_start_positions[sentence_ind][word_ind] - sentence_start_positions[sentence_ind]
                        current_word_start_position_in_simplified_sentence = words_start_positions_in_simplified_sentence[word_ind]
                        #print(current_word)
                        suggested_word = current_word
                        if model == "bert":
                            suggested_word = bert_model(sentences[sentence_ind], current_word, current_word_start_position_in_initial_sentence, complexity_method)
                        elif model == "synonym":
                            suggested_word = synonym_model(current_word, complexity_method)

                        #print(suggested_word)
                        simplified_sentence = simplified_sentence[:current_word_start_position_in_simplified_sentence] + suggested_word + simplified_sentence[current_word_start_position_in_simplified_sentence + len(current_word):]
                        simplification_offset = len(suggested_word) - len(current_word)

                        for new_word_ind in range(word_ind + 1, len(words[sentence_ind])):
                            words_start_positions_in_simplified_sentence[new_word_ind] += simplification_offset

        simplified_sentences.append(simplified_sentence)
        print(sentences[sentence_ind])
        print(simplified_sentence)
        print()

    return simplified_sentences

In [85]:
print(simplify(text_location="/tmp/NEREL-master/NEREL-v1.1/train/023.txt", model="synonym", complexity_method="len"))

Финансовым директором социальной сети Facebook назначен 39-летний Дэвид Эберсман (David Ebersman), сообщает The Wall Street Journal.
Финансовым директором светской сети Facebook назначен 39-летний Дэвид Эберсман (David Ebersman), пишет The Wall Street Journal.

На работу в Facebook он выйдет в сентябре.
На работу в Facebook он выйдет в сентябре.

Ранее Эберсман был финансовым директором биотехнологической компании Genentech.
Ранее Эберсман был финансовым директором биотехнологической компании Genentech.

Эберсман подчеркнул, что видит много общего между Facebook и Genentech.
Эберсман указал, что видит много общего между Facebook и Genentech.

В частности, это две быстрорастущие компании с сильной корпоративной культурой.
В частности, это две быстрорастущие компании с сильной общей культурой.

Также он заявил, что Facebook ожидает 70-процентное увеличение выручки в 2009 году.
Также он заявил, что Facebook ожидает 70-процентное увеличение выручки в 2009 году.

В компании Genentech Дэвид 

#Assessment

In [74]:
directory = "/tmp/NEREL-master/NEREL-v1.1/train/"
file_list = os.listdir(directory)
txt_file_list = [file_name for file_name in file_list if file_name.endswith(".txt")]

random.shuffle(txt_file_list)

sample_size = 20
txt_file_list = txt_file_list[:sample_size]

simple_dir = '/content/drive/My Drive/simplifications'
models = [("baseline", ""), ("bert", "len"), ("bert", "frequency"), ("synonym", "len"), ("synonym", "frequency")]

In [75]:
print(txt_file_list) #['24632_text.txt']

['19054_text.txt', '12522_text.txt', '100986_text.txt', '39538_text.txt', '463.txt', '37719_text.txt', '20160_text.txt', '82299_text.txt', '149704_text.txt', '25896_text.txt', '105566_text.txt', '509121_text.txt', '38217_text.txt', '19358_text.txt', '133604_text.txt', '141316_text.txt', '25518_text.txt', '141309_text.txt', '531.txt', '24647_text.txt']


In [ ]:
if not os.path.exists(os.path.join(simple_dir, 'original')):
    os.makedirs(os.path.join(simple_dir, 'original'))

for file_name in txt_file_list:
    for model, complexity_method in models:
        file_path = os.path.join(directory, file_name)
        if os.path.isfile(file_path):
            simplified_sentences = simplify(file_path, model, complexity_method)
            if not os.path.exists(os.path.join(simple_dir, model, complexity_method)):
                os.makedirs(os.path.join(simple_dir, model, complexity_method))
            write_list_to_file(simplified_sentences, os.path.join(simple_dir, model, complexity_method, file_name))

In [67]:
def count_syllables(word):
    word = re.sub(r'[^аеёиоуыэюя]', '', word.lower())
    return len(word)

def asw(sentences): # average number of syllables in the word
    total_avg_syllables = 0
    total_sentences = 0
    for sentence in sentences:
        words = re.findall(r'\b\w+\b', sentence)
        if len(words) != 0:
            total_sentences += 1
            total_syllables = 0
            for word in words:
                total_syllables += count_syllables(word)
            total_avg_syllables += total_syllables / len(words)
    return total_avg_syllables / total_sentences

In [ ]:
bert_embed_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
bert_embed_model = AutoModel.from_pretrained("bert-base-uncased")

def bert_cos_dist(original_sentences, simple_sentences):
    avg_dist = 0
    for i in range(min(len(simple_sentences), len(original_sentences))):
        sentence1 = original_sentences[i]
        sentence2 = simple_sentences[i]

        encoded_input = bert_embed_tokenizer([sentence1, sentence2], padding=True, truncation=True, return_tensors='pt')
        with torch.no_grad():
            output = bert_embed_model(**encoded_input)

        sentence1_embedding = output.last_hidden_state[0, 0, :]
        sentence2_embedding = output.last_hidden_state[1, 0, :]

        dist = cosine_distances(sentence1_embedding.reshape(1, -1), sentence2_embedding.reshape(1, -1)).item()
        avg_dist += dist    
    return avg_dist / min(len(simple_sentences), len(original_sentences))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
sentences1 = ["Мне нравятся коты", "I love you"]
sentences2 = ["Я люблю собак", "Je t'aime"]
distance = bert_cos_dist(sentences1, sentences2)
print("Cosine distance:", distance)

Cosine distance: 0.0805254876613617


In [68]:
def assess_model(simple_dir, model, file_list, metric):
    avg_metric = 0
    files_num = 0
    for file_name in file_list:
        simple_location = os.path.join(simple_dir, model[0], model[1], file_name)
        simple_sentences = read_list_from_file(simple_location)
        if metric == 'asw':
            if len(simple_sentences) != 0:
                files_num += 1
                avg_metric += asw(simple_sentences)
        elif metric == 'bert_cos_dist':
            original_location = os.path.join(simple_dir, 'original', file_name)
            original_sentences = read_list_from_file(original_location)
            print(simple_sentences)
            print(original_sentences)
            avg_metric += bert_cos_dist(simple_sentences, original_sentences)
    return avg_metric/files_num

In [70]:
for model in models:
    for metric in ['asw']: #"bert_cos_dist"
        res = assess_model(simple_dir, model, txt_file_list, metric)
        print("Model: " + str(model[0]))
        print("Complexity method: " + str(model[1]))
        print("Metric: " + metric)
        print("Result: " + str(res))
        print()

Model: baseline
Complexity method: 
Metric: asw
Result: 2.4154692467190717

Model: bert
Complexity method: len
Metric: asw
Result: 2.1144077338760026

Model: bert
Complexity method: frequency
Metric: asw
Result: 2.3621361878725375

Model: synonym
Complexity method: len
Metric: asw
Result: 2.2702403694104114

Model: synonym
Complexity method: frequency
Metric: asw
Result: 2.441953394574341

